Setup commands and imports

In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import mediapipe as mp
import cv2
import torch
from cnn import CNN
import re
import sklearn

In [23]:
for folder in ['train_data', 'val_data', 'test_data']:
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        if filename[:7] == 'Sitting':
            new_name = filename[:13] + filename[15:]
            rename_path = os.path.join(folder, new_name)
            os.rename(filepath, rename_path)
            # print(rename_path)

Evaluate performance of best model

In [2]:
from evaluate import evaluate


print('Fully-connected performance')
print('=' * 30)
evaluate('fc')

print()

print('CNN performance')
print('=' * 30)
evaluate('cnn')

Fully-connected performance
Validation accuracy = 0.7674
Test accuracy = 0.7299

CNN performance
Validation accuracy = 0.7859
Test accuracy = 0.7760


Save raw MediaPose data into its own folder.

In [2]:
import os
import numpy as np
import mediapipe as mp
import cv2


mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3, model_complexity=2)

for folder in os.listdir('yoga'):
    output = np.empty((0, 69), dtype=float)
    folder_path = os.path.join('yoga', folder)

    for filename in os.listdir(folder_path):
        body = np.empty(0, dtype=float)
        filepath = os.path.join(folder_path, filename)
        try:
            img = cv2.imread(filepath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        except:
            continue

        result = pose.process(img)

        if result.pose_landmarks:
            for i, point in enumerate(result.pose_landmarks.landmark):
                if i in range(1, 11):
                    continue

                xyz = np.array([point.x, point.y, point.z])
                body = np.append(body, xyz)
            
            output = np.vstack((output, body))

    print(f'{folder}\t{output.shape}')
    np.save(f'data_raw/{folder}.npy', output, allow_pickle=True)

Akarna_Dhanurasana	(66, 69)
Bharadvaja's_Twist_pose_or_Bharadvajasana_I_	(77, 69)
Boat_Pose_or_Paripurna_Navasana_	(356, 69)
Bound_Angle_Pose_or_Baddha_Konasana_	(236, 69)
Bow_Pose_or_Dhanurasana_	(205, 69)
Bridge_Pose_or_Setu_Bandha_Sarvangasana_	(254, 69)
Camel_Pose_or_Ustrasana_	(277, 69)
Cat_Cow_Pose_or_Marjaryasana_	(375, 69)
Chair_Pose_or_Utkatasana_	(286, 69)
Child_Pose_or_Balasana_	(269, 69)
Cobra_Pose_or_Bhujangasana_	(709, 69)
Cockerel_Pose	(135, 69)
Corpse_Pose_or_Savasana_	(290, 69)
Cow_Face_Pose_or_Gomukhasana_	(264, 69)
Crane_(Crow)_Pose_or_Bakasana_	(270, 69)
Dolphin_Plank_Pose_or_Makara_Adho_Mukha_Svanasana_	(45, 69)
Dolphin_Pose_or_Ardha_Pincha_Mayurasana_	(64, 69)
Downward-Facing_Dog_pose_or_Adho_Mukha_Svanasana_	(275, 69)
Eagle_Pose_or_Garudasana_	(268, 69)
Eight-Angle_Pose_or_Astavakrasana_	(173, 69)
Extended_Puppy_Pose_or_Uttana_Shishosana_	(74, 69)
Extended_Revolved_Side_Angle_Pose_or_Utthita_Parsvakonasana_	(504, 69)
Extended_Revolved_Triangle_Pose_or_Utthita_Tri

In [4]:
import yoga_dataset
import os
import numpy as np
import math


count = 0
train_prop, val_prop, test_prop = 0.7, 0.1, 0.2
pose2idx = {pose: idx for pose, idx in zip(os.listdir('yoga'), range(len(os.listdir('yoga'))))}

train_data, val_data, test_data = [], [], []
train_labels, val_labels, test_labels = [], [], []

for filename in os.listdir('data'):
    filepath = os.path.join('data', filename)
    arr = np.load(filepath)
    train_amt = math.floor(train_prop * arr.shape[0])
    val_amt = train_amt + math.floor(val_prop * arr.shape[0])

    train_data.append(arr[:train_amt])
    val_data.append(arr[train_amt:val_amt])
    test_data.append(arr[val_amt:])

    pose_idx = pose2idx[filename[:-4]]
    labels = np.ones(arr.shape[0]) * pose_idx

    train_labels.append(labels[:train_amt])
    val_labels.append(labels[train_amt:val_amt])
    test_labels.append(labels[val_amt:])

    # for i, body in enumerate(arr):
    #     assert body.shape == (23, 3), 'wrong shape of array'
        
    #     if i < train_amt:
    #         np.save(f'train_data/{filename[:-4]}{i}.npy', body, allow_pickle=True)
    #     elif i < train_amt + val_amt:
    #         np.save(f'val_data/{filename[:-4]}{i}.npy', body, allow_pickle=True)
    #     else:
    #         np.save(f'test_data/{filename[:-4]}{i}.npy', body, allow_pickle=True)

np.save('train.npy', np.concatenate(train_data))
np.save('train_labels.npy', np.concatenate(train_labels))
np.save('val_labels.npy', np.concatenate(val_labels))
np.save('val.npy', np.concatenate(val_data))
np.save('test.npy', np.concatenate(test_data))
np.save('test_labels.npy', np.concatenate(test_labels))

print(f'amount of train data: {np.load("train.npy").shape[0]}')
print(f'amount of val data: {np.load("val.npy").shape[0]}')
print(f'amount of test data: {np.load("test.npy").shape[0]}')

amount of train data: 11550
amount of val data: 1617
amount of test data: 3385


In [4]:
import torch
from cnn import CNN


input = torch.randn(10, 3, 23)
model = CNN([3, 16, 32, 64, 64])
out = model(input)
print(out.shape)

torch.Size([10, 82])


In [5]:
import re
import os


for file in os.listdir('val_data'):
    posename = re.split('[0-9]', file, 1)[0]
    print(f'pose = {posename}')

pose = Akarna_Dhanurasana
pose = Akarna_Dhanurasana
pose = Akarna_Dhanurasana
pose = Akarna_Dhanurasana
pose = Akarna_Dhanurasana
pose = Akarna_Dhanurasana
pose = Bharadvaja's_Twist_pose_or_Bharadvajasana_I_
pose = Bharadvaja's_Twist_pose_or_Bharadvajasana_I_
pose = Bharadvaja's_Twist_pose_or_Bharadvajasana_I_
pose = Bharadvaja's_Twist_pose_or_Bharadvajasana_I_
pose = Bharadvaja's_Twist_pose_or_Bharadvajasana_I_
pose = Bharadvaja's_Twist_pose_or_Bharadvajasana_I_
pose = Bharadvaja's_Twist_pose_or_Bharadvajasana_I_
pose = Boat_Pose_or_Paripurna_Navasana_
pose = Boat_Pose_or_Paripurna_Navasana_
pose = Boat_Pose_or_Paripurna_Navasana_
pose = Boat_Pose_or_Paripurna_Navasana_
pose = Boat_Pose_or_Paripurna_Navasana_
pose = Boat_Pose_or_Paripurna_Navasana_
pose = Boat_Pose_or_Paripurna_Navasana_
pose = Boat_Pose_or_Paripurna_Navasana_
pose = Boat_Pose_or_Paripurna_Navasana_
pose = Boat_Pose_or_Paripurna_Navasana_
pose = Boat_Pose_or_Paripurna_Navasana_
pose = Boat_Pose_or_Paripurna_Navasana_
